In [1]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, AutoTokenizer, AutoModelForCausalLM
import torch

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
model = GPT2LMHeadModel.from_pretrained("output/checkpoint-13000").to(device)
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/ko-gpt-trinity-1.2B-v0.5",
  bos_token='<s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [4]:
tokenizer.add_special_tokens({'additional_special_tokens': ["<yun>"]})
model.resize_token_embeddings(len(tokenizer))

Embedding(51201, 1920)

In [35]:
sentence = '<s>떠나간 봄날의 설레는 기다림\n'
inputs = tokenizer.encode(sentence, add_special_tokens=True, return_tensors='pt').to(device)

with torch.no_grad():
    output = model.generate(
        input_ids=inputs,
        temperature=0.6, # 생성 다양성 조절
        min_length=64, # 생성되는 문장의 최소 길이
        max_length=256, # 생성되는 문장의 최대 길이
        repetition_penalty=1.2, # 반복을 줄이는 패널티
        do_sample=True, # 샘플링 기반 생성 활성화
        early_stopping=True, # EOS token을 만나면 조기 종료
        eos_token_id=tokenizer.eos_token_id,
        top_k=10,
        top_p=0.95
    )

generated_sequence = output[0].tolist()
text = tokenizer.decode(generated_sequence, skip_special_tokens=False)

In [36]:
text = text.replace("<yun> ", "\n").replace("<s> ", "").replace("</s>", "")

In [37]:
print(text)

떠나간 봄날의 설레는 기다림
그리움으로 다시 찾아온 봄의 향기

그 옛날 보릿고개 넘을 때 
마지막 이었던 매화꽃 피던 날
가슴에 살며시 물결치는 그리움

새봄이 돌아오면 꽃잎은 더 푸르고
살랑이는 봄바람 불 때면 옛 추억에
잠겨오는 아련한 첫사랑의 연가

목련의 고귀한 순결함처럼
그대의 따사한 눈길 속에 피는
곱디 고운 사랑의 미로

춘삼월엔 맘 속 깊숙하게 파고드는
아름다운 사랑 노래하고픈데
보리피리 만들어 부르면서
첫사랑의 향연 그리워하네
